In [ ]:
!pip install kagglehub

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
# Load dataset
# customer_path = os.path.join(path, "customer.csv")
# loan_path = os.path.join(path, "loan.csv")

# customer = pd.read_csv(customer_path)
# loan = pd.read_csv(loan_path)
# df = pd.merge(customer, loan, on='customer_id', how='inner').drop("loan_amount", axis=1)
df =pd.read_csv("/home/chun/Downloads/cs-267/project/join_df.csv")

In [ ]:


# Assuming the dataframe `df` is already loaded and contains 'funded_amount' as the target
# Preprocess the dataset
df = df.dropna()  # Handle missing data
df = pd.get_dummies(df, drop_first=True)  # Convert categorical columns to one-hot encoding

# Feature and target split
X = df.drop('funded_amount', axis=1).values  # Features
y = df['funded_amount'].values  # Target

# Reshape data for RNN (adding a time-step dimension)
# Assuming each sample is treated as a single time-step sequence
X = np.expand_dims(X, axis=1)  # Shape: (samples, time-steps, features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)




In [ ]:
len(df.columns)

6545

In [ ]:
def rmse_loss(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Build the RNN model
model = Sequential([
    LSTM(50, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(25, activation='tanh', return_sequences=False),
    Dropout(0.3),
    Dense(12, activation='relu'),
    Dense(1)  # Single output neuron for regression
])
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

# Compile the model
model.compile(optimizer='adam', loss=rmse_loss, metrics=['mae'])  # Use MSE loss for regression

# Train the model
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    verbose=1)

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Mean Absolute Error: {test_mae:.2f}")

# Predict and analyze results
y_pred = model.predict(X_test)
print(f"Predicted loan amounts (first 5): {y_pred[:5].flatten()}")

/home/chun/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 18150.6309 - mae: 15535.3115 - val_loss: 18528.1855 - val_mae: 15924.1387
Epoch 2/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 17990.1895 - mae: 15428.2383 - val_loss: 18426.9180 - val_mae: 15806.2666
Epoch 3/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 17969.2578 - mae: 15330.3145 - val_loss: 18280.6172 - val_mae: 15635.5449
Epoch 4/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 17918.8770 - mae: 15268.1729 - val_loss: 18091.0996 - val_mae: 15413.6494
Epoch 5/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 17562.3691 - mae: 14864.0762 - val_loss: 17861.8594 - val_mae: 15144.0566
Epoch 6/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 17136.2207 - mae: 14454.5713 - val_loss: 17595.1016 - val_mae: 14830.5400
Epoch 7/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 17003.6484 - mae: 14318.4121 - val_loss: 17295.4277 - val_mae: 14478.6885
Epoch 8/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 16

In [ ]:
model.summary()